## Implementing Agentic RAG using Langchain
* https://medium.com/the-ai-forum/implementing-agentic-rag-using-langchain-b22af7f6a3b5
* 아... 모르겠음 ㅋ

In [3]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search langchain-community
!pip install -qU faiss-cpu pymupdf python-dotenv

In [17]:
from uuid import uuid4
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = f"agentic-rag-{uuid4().hex[0:8]}"

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

docs = ArxivLoader(query="Retrieval Augmented Generation", max_results=5).load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)
chunked_documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
)
retriever = faiss_vectorstore.as_retriever()

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: 

In [19]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """\
Use the following context to answer the user's query in Korean. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

template = ChatPromptTemplate.from_template(rag_prompt)
template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following context to answer the user's query in Korean. If you cannot answer the question, please respond with 'I don't know'.\n\nQuestion:\n{question}\n\nContext:\n{context}\n"))])

In [20]:
from langchain_openai import ChatOpenAI

open_chat_model = ChatOpenAI(model="gpt-4o")

In [21]:
# chain
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question") }
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": template | open_chat_model, "context": itemgetter("context")}
)
chain

{
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x3619a3550>),
  question: RunnableLambda(itemgetter('question'))
}
| RunnableAssign(mapper={
    context: RunnableLambda(itemgetter('context'))
  })
| {
    response: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following context to answer the user's query in Korean. If you cannot answer the question, please respond with 'I don't know'.\n\nQuestion:\n{question}\n\nContext:\n{context}\n"))])
              | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x127da9720>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x127dab4f0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy=''),
    conte

### Ask Query

In [51]:
await chain.ainvoke({"question": "RAG 가 무엇인가요?"})

{'response': AIMessage(content='RAG는 Retrieval-Augmented Generation의 약자로, 정보 검색 프로세스를 통합하여 대형 언어 모델(LLM)의 정적 한계를 해결하는 방법론입니다. 이는 최신 외부 정보를 동적으로 통합함으로써 LLM의 정확성과 신뢰성을 향상시키는 것을 목표로 합니다. RAG는 주로 텍스트 도메인에 초점을 맞추며, LLM이 실제 데이터를 사용하여 더 정확하고 신뢰할 수 있는 출력을 생성하도록 돕습니다. RAG는 사전 검색, 검색, 검색 후 처리, 생성의 네 가지 범주로 조직되며, 각 단계에서 성능을 최적화하기 위한 다양한 기법과 평가 방법이 포함됩니다.', response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 2747, 'total_tokens': 2900}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_43dfabdef1', 'finish_reason': 'stop', 'logprobs': None}, id='run-cff8b354-2b4a-4832-b66b-da3b0ca174bc-0'),
 'context': [Document(page_content='RGB (Chen et al., 2023b)\nImpact of RAG on LLMs\nNoise Robustness\nAccuracy\nSynthetic Dataset including English and Chinese\nNegative Rejection\nRejection Rate\nInformation Integration\nAccuracy\nCounterfactual Robustness\nError Detection Rate\nError Correction Rate\nTable 3: The Comparison of Different RAG Evaluation Frameworks\n8.1\nRetrieval-base

Duck Duck Go Web Search
* https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search

In [52]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langgraph.prebuilt import ToolExecutor

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun(),
]

tool_executor = ToolExecutor(tool_belt)

In [53]:
from langchain_core.utils.function_calling import convert_to_openai_function

model = ChatOpenAI(model="gpt-4o", temperature=0)
functions = [convert_to_openai_function(f) for f in tool_belt]
model = model.bind_functions(functions)

In [54]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [57]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

def call_tool(state):
    last_message = state["messages"][-1]
    
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    )
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)

    return {"messages": [function_message]}


In [63]:
# build workflow

from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.nodes

{'agent': agent(recurse=True), 'action': action(recurse=True)}

In [64]:
workflow.set_entry_point("agent")

In [65]:
def should_continue(state):
    last_message = state["messages"][-1]

    if "function_call" not in last_message.additional_kwargs:
        return "end"
    
    return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

In [66]:
workflow.add_edge("action", "agent")

In [67]:
app = workflow.compile()
app

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 0x1048dfd70>, skip_none=False, mapper=_get_state_key(recurse=False))]), ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__', skip_none=False, mapper=None)])]), 'agent': PregelNode(config={'tags': []}, channels={'messages': 'messages'}, triggers=['start:agent', 'action'], mapper=functools.partial(<function _coerce_state at 0x110347760>, <class '__main__.AgentState'>), writers=[ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', value='agent', skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 0x1048dfd70>, skip_none=False, mapper=_get_state_key(recurse=False))]), _route(recurse=True, _is_channel_writer=True)])

In [70]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="LLM에서 RAG는 무엇인가요? 언제 언급되기 시작했나요?")]}
response = app.invoke(inputs)
print(response['messages'][-1].content)

RAG는 "Retrieval-Augmented Generation"의 약자로, 대형 언어 모델(LLM)에서 정보를 생성할 때 외부 데이터베이스나 문서에서 관련 정보를 검색하여 이를 활용하는 방법론을 의미합니다. RAG는 주로 다음과 같은 두 가지 주요 단계를 포함합니다:

1. **Retrieval (검색)**: 사용자의 질문이나 요청에 대해 관련된 문서를 외부 데이터베이스나 문서 집합에서 검색합니다. 이 단계에서는 검색 엔진이나 정보 검색 알고리즘을 사용하여 관련성이 높은 문서를 찾습니다.

2. **Generation (생성)**: 검색된 문서를 바탕으로 사용자의 질문에 대한 답변을 생성합니다. 이 단계에서는 대형 언어 모델이 검색된 정보를 활용하여 보다 정확하고 풍부한 답변을 생성합니다.

RAG는 특히 다음과 같은 상황에서 유용합니다:
- **정보의 최신성**: 모델이 학습된 이후에 발생한 최신 정보를 반영할 수 있습니다.
- **전문성**: 특정 도메인에 대한 깊이 있는 지식이 필요할 때, 해당 도메인의 문서를 검색하여 보다 정확한 답변을 제공할 수 있습니다.
- **모델의 한계 보완**: 모델이 학습 데이터에 포함되지 않은 정보를 제공해야 할 때, 외부 문서를 검색하여 이를 보완할 수 있습니다.

RAG는 최근 몇 년간 언급되기 시작했으며, 특히 2020년대 초반부터 주목받기 시작했습니다. 이는 대형 언어 모델의 성능이 향상됨에 따라, 단순한 텍스트 생성에서 벗어나 보다 정확하고 신뢰할 수 있는 정보를 제공하기 위한 방법론으로 발전해왔기 때문입니다.


In [71]:
question = "Retrieval Augmented Generation 의 메인 저자는 누구인가요? 그리고 어떤 대학을 나왔나요"

inputs = {"messages" : [HumanMessage(content=question)]}
response = app.invoke(inputs)
print(response['messages'][-1].content)

Retrieval Augmented Generation (RAG) is a topic that has been explored by multiple researchers. Here are some of the main authors from notable papers on the subject:

1. **Huayang Li, Yixuan Su, Deng Cai, Yan Wang, Lemao Liu** - Authors of "A Survey on Retrieval-Augmented Text Generation" (Published: 2022-02-13).
2. **Dian Jiao, Li Cai, Jingsheng Huang, Wenqiao Zhang, Siliang Tang, Yueting Zhuang** - Authors of "DuetRAG: Collaborative Retrieval-Augmented Generation" (Published: 2024-05-12).
3. **Raviteja Anantha, Tharun Bethi, Danil Vodianik, Srinivas Chappidi** - Authors of "Context Tuning for Retrieval Augmented Generation" (Published: 2023-12-09).

To determine the educational background of these authors, I would need to look up each individual author. If you have a specific author in mind, please let me know, and I can search for their educational background.
